<a href="https://colab.research.google.com/github/rodrihgh/white-nightmare/blob/master/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# White Nightmare

Welcome to the interactive version of my project White Nightmare.

You can also view the code on
[GitHub](https://github.com/rodrihgh/white-nightmare)
and read the back story on my website,
[Science of Uselessness](https://rodrihgh.github.io/projects/white-nightmare/).

## Imports

In [0]:
import numpy as np
from numpy.random import normal, binomial, choice
from scipy import signal, ndimage
import warnings

import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML


## Helper Functions

That is where the algorithm magic happens.


In [0]:
#@title

def time2samples(t, rate):
    """ Convert time to number of samples at a certain rate.
    
    Args:
        t (float): Time in seconds
        rate (float): Sampling rate.

    Returns:
        int: Number of samples for the given time at the given rate.

    """
    return int(round(t * rate))


def frameshow(frame, v=None, cmap='gist_gray'):
    """ Plot one frame
    
    Args:
        frame (array_like): Frame to plot.
        v (iterable, optional): Lower and upper values for the colour axis.
            If None, use the min and max values
        cmap (string, optional): Color mapping used by matplotlib

    Returns:
        Figure and Image objects.

    """
    f = plt.figure()

    if v is None:
        vmin = frame.min()
        vmax = frame.max()
    else:
        vmin = v[0]
        vmax = v[1]

    im = plt.imshow(frame, vmin=vmin, vmax=vmax, cmap=cmap)

    aspect_ratio = frame.shape[1] / frame.shape[0]

    margin = 0.85

    plt.axis('off')
    plt.tight_layout(pad=0)
    w, h = f.get_size_inches()
    f.set_size_inches([margin * w, margin * w / aspect_ratio])

    return f, im


def animate(frames, fps=24, cmap='gist_gray', v=None):
    """ Create an animation from a frame array
    
    Args:
        frames (array_like): Frame array. 
        fps (float, optional): Rate of frames per second.
        cmap (string, optional): Colour mapping used by matplotlib.
        v (iterable, optional): Lower and upper values for the colour axis.
            If None, use the min and max values

    Returns:
        Animation object.

    """
       

    f, im = frameshow(frames[-1, ], v=v, cmap=cmap)

    def update(num, data, img):
        img.set_data(data[num, ])
        return img,

    im_ani = animation.FuncAnimation(f, update, frames.shape[0], fargs=(frames, im), interval=1000 / fps, blit=True)

    return im_ani


def resample(array, factors, order=3, mode='mirror'):
    """ Resample multidimensional arrays.
    
    Args:
        array (array_like): Input array.
        factors (iterable): Resampling factor for each axis.
        order (int, optional): Order of spline interpolation.
        mode (string, optional): Extension mode of the input beyond its boundaries
    
    Returns:
        array_like: Resampled array.
        
    """
    return ndimage.zoom(array, factors, order=order, mode=mode)


def ar_filter(array, alpha, axis):
    """ Filter a multidimensional array according to an AR(1) process.
    
    Args:
        array (array_like): Input array.
        alpha (float): Parameter of AR(1).
        axis (int): Axis where filtering is applied.

    Returns:
        array_like: Filtered array.
    """

    b = [1 - alpha]
    a = [1, -alpha]

    ar = signal.lfilter(b, a, array, axis=axis)

    return ar


def ar_gain(alpha):
    """ Calculate ratio between the standard deviation
        of the noise term in an AR(1) process and the resultant
        standard deviation of the AR(1) process.
    
    Args:
        alpha (float): Parameter of AR(1). 

    Returns:
        float: Ratio between std of noise term and std of AR(1)

    """
    return np.sqrt((1 + alpha) / (1 - alpha))


def db2mag(db):
    """ Decibel to Magnitude conversion.
    
    Args:
        db (float): Decibel level.

    Returns:
        float: Magnitude.

    """
    return 10 ** (db / 20)


def db2power(db):
    """ Decibel to power conversion.
    
    Args:
        db (float): Decibel level.

    Returns:
        float: Power.

    """
    
    return 10 ** (db / 10)


def mag2db(mag):
    """ Magnitude to decibel conversion.
    
    Args:
        mag (float): Magnitude level.

    Returns:
        float: Decibel.

    """
    
    return 20 * np.log10(mag)


def get_alpha(attenuation, lag):
    """Calculate the alpha of AR(1) so that the attenuation condition at the specified lag is fulfilled.
    
    Args:
        attenuation (float): Desired attenuation at lag in dB. Must be positive.
        lag (float): Reference lag period in samples. Must be greater or equal than 2.
        
    Returns:
        float: Calculated alpha.
    
    """
    tol = 0.01

    if lag < 2:
        raise ValueError(f"Lag must be greater or equal than 2. Input value is {lag}")
    elif attenuation <= 0:
        raise ValueError(f"Attenuation must be positive. Input value is {attenuation}")

    w = 2 * np.pi / lag
    rho = db2power(attenuation)

    cos_w = np.cos(w)
    b = (rho - cos_w) / (rho - 1)
    alpha = b - np.sqrt(b ** 2 - 1)

    if alpha >= (1 - tol):
        warnings.warn(f"Alpha value equals {alpha}. This value is too close to one and may cause instability",
                      RuntimeWarning)

    return alpha


def background(shape, white, black, std, p=0.5, upsampling=None, order=3):
    """ Generate noisy background as a sum of binomial and gaussian
    
    Args:
        shape (iterable): shape of the generated background
        white (float): Numeric value of white
        black (float): Numeric value of black
        std (float): Standard deviation of gaussian term
        p (float, optional):  Probability of the binomial term
        upsampling (iterable, optional): Upsampling factors for each axis.
            Default is None, meaning no upsampling
        order (int, optional): Interpolation order for upsampling, if applied.
            Defaults to 3.
    
    Returns:
        array_like: Noisy background.
    """

    bg = (white - black) * binomial(1, p=p, size=shape) + normal(loc=black, scale=std, size=shape)
    if upsampling is not None:
        bg = resample(bg, upsampling, order=order)
    return bg


def trunc_norm(loc=0.0, scale=1.0, a=-1.0, b=1.0, size=None):
    """ Generate samples according to a truncated normal distribution.
    
    Args:
        loc (float, optional): Mean of the distribution
        scale (float, optional): Standard deviation.
        a (float, optional): Lower bound.
        b (float, optional): Upper bound.
        size (float, optional): Size of the returned samples
    
    Returns:
        array_like: Array of shape `size` whose values follow a truncated normal distribution.
        
    """
    x = normal(loc, scale, size)

    if size is None:
        while not a <= x <= b:
            x = normal(loc, scale, size)
    else:
        invalid = np.where(np.logical_or(x < a, b < x))
        invalid_count = len(invalid[0])
        while invalid_count > 0:
            x[invalid] = normal(loc, scale, size=invalid_count)
            invalid = np.where(np.logical_or(x < a, b < x))
            invalid_count = len(invalid[0])

    return x


def wilson_algorithm(maze_height, maze_width, root_cell="SE"):
    """ Graph tree generation of a maze using Wilson algorithm.
    
    Args:
        maze_width (int): Width of the squared maze in cells.
        maze_height(int): Height of the squared maze in cells.
        root_cell (string, optional): Selection of the initial cell.
    
    Returns:
        tuple: Coordinates of root cell.
        list: Nested list where children cells are indexed by their root's coordinates.
        
    """
    starting_values = {"NW": (0, 0),
                       "NE": (0, maze_width - 1),
                       "SW": (maze_height - 1, 0),
                       "SE": (maze_height - 1, maze_width - 1),
                       "CENTER": (maze_height // 2, maze_width // 2),
                       "RANDOM": (choice(maze_height), choice(maze_width))}

    if root_cell not in starting_values:
        raise ValueError(f"Unrecognized root cell {root_cell}."
                         f"Permitted values are {tuple(starting_values.keys())}")
    else:
        root = starting_values[root_cell]

    compass = ((-1, 0), (0, 1), (1, 0), (0, -1))

    def get_neighbors(index):
        neighbor_list = []
        for c in compass:
            cell_i = index[0] + c[0]
            cell_j = index[1] + c[1]
            if 0 <= cell_i < maze_height and 0 <= cell_j < maze_width:
                neighbor_list.append((cell_i, cell_j))
        return neighbor_list

    children = [[[] for _ in range(maze_width)] for _ in range(maze_height)]
    maze_cells = np.zeros((maze_height, maze_width), dtype=np.bool)
    maze_cells[root] = True

    while not np.all(maze_cells):
        visit_i, visit_j = np.where(np.logical_not(maze_cells))
        visit_index = choice(len(visit_i))
        cell2visit = (visit_i[visit_index], visit_j[visit_index])
        visited_path = [cell2visit]
        visited_cells = np.zeros_like(maze_cells)
        visited_cells[cell2visit] = True
        while not maze_cells[cell2visit]:
            neighbors = get_neighbors(cell2visit)
            i_neighbor = choice(len(neighbors))
            cell2visit = neighbors[i_neighbor]
            if not visited_cells[cell2visit]:
                visited_path.append(cell2visit)
                visited_cells[cell2visit] = True
            else:
                loop_cell = cell2visit
                cell2visit = visited_path[-1]
                while cell2visit != loop_cell:
                    visited_cells[cell2visit] = False
                    del visited_path[-1]
                    cell2visit = visited_path[-1]
        reverse_path = visited_path[::-1]

        maze_cells = np.logical_or(maze_cells, visited_cells)
        for n in range(len(reverse_path) - 1):
            curr_cell = reverse_path[n + 1]
            prev_cell = reverse_path[n]
            children[prev_cell[0]][prev_cell[1]].append(curr_cell)

    return root, children


def render_maze(root, children, alpha, std, black=0.0, white=1.0, history=True):
    """ Render maze walls and cells according to an autoregressive model.
    
    Args:
        root (iterable): Coordinates of root cell.
        children (list): Nested list where children cells are indexed by their root's coordinates.
        alpha (float): Parameter of the AR(1) process
        std (float): Standard deviation of the AR(1) process.
        black (float, optional): Value of the black color associated to walls.
        white (float, optional): Value of the white color associated to cells.
        history (bool, optional): Boolean indicating whether to keep a frame-by-frame record of the rendering.
        
    Returns:
        numpy.ndarray: Rendered maze.
        numpy.ndarray, optional: If history is set, this is also returned as a frame by frame mask.
    
    """

    sigma = std * ar_gain(alpha)

    def x_ar(*x_past, loc=black):
        x_curr = alpha * np.sum(x_past) + (1 - len(x_past) * alpha) * normal(loc=loc, scale=sigma)
        return x_curr

    edge_height = len(children) * 2
    edge_width = len(children[0]) * 2

    current_branches = [root]

    rendered_maze = np.zeros(shape=(edge_height, edge_width), dtype=np.float)
    maze_mask = np.zeros(shape=rendered_maze.shape, dtype=np.bool) if history else None

    i_root = 2 * root[0]
    j_root = 2 * root[1]

    for i in range(-1, 2):
        for j in range(-1, 2):
            if history:
                maze_mask[i_root + i, j_root + j] = True
            if i == 0 and j == 0:
                rendered_maze[i_root, j_root] = normal(loc=white, scale=sigma)
            else:
                rendered_maze[i_root + i, j_root + j] = normal(loc=black, scale=sigma)

    maze_history = maze_mask[None, ] if history else None

    while len(current_branches) > 0:
        future_branches = []
        for branch in current_branches:
            for child in children[branch[0]][branch[1]]:

                # Calculate position of cells to render
                direction = (child[0] - branch[0], child[1] - branch[1])

                mid_cell = (branch[0] + child[0], branch[1] + child[1])
                end_cell = (2 * child[0], 2 * child[1])

                void1a = (2 * child[0] + direction[1], 2 * child[1] + direction[0])
                void1b = (2 * child[0] - direction[1], 2 * child[1] - direction[0])
                void2a = (2 * child[0] + direction[0] + direction[1], 2 * child[1] + direction[1] + direction[0])
                void2b = (2 * child[0] + direction[0] - direction[1], 2 * child[1] + direction[1] - direction[0])
                end_void = (2 * child[0] + direction[0], 2 * child[1] + direction[1])

                # Fetch value of already rendered cells
                w0 = rendered_maze[2 * branch[0], 2 * branch[1]]

                b0a = rendered_maze[branch[0] + child[0] + direction[1], branch[1] + child[1] + direction[0]]
                b0b = rendered_maze[branch[0] + child[0] - direction[1], branch[1] + child[1] - direction[0]]

                # Calculate value of newly rendered cells
                w1 = x_ar(w0, loc=white)
                w2 = x_ar(w1, loc=white)

                b1a = x_ar(b0a, loc=black)
                b1b = x_ar(b0b, loc=black)
                b2a = x_ar(b1a, loc=black)
                b2b = x_ar(b1b, loc=black)
                b3 = x_ar(b2a, b2b, loc=black)

                # Assign rendered values
                rendered_maze[mid_cell] = w1
                rendered_maze[end_cell] = w2

                rendered_maze[void1a] = b1a
                rendered_maze[void1b] = b1b
                rendered_maze[void2a] = b2a
                rendered_maze[void2b] = b2b
                rendered_maze[end_void] = b3

                if history:

                    maze_mask[mid_cell] = True
                    maze_mask[end_cell] = True

                    maze_mask[void1a] = True
                    maze_mask[void1b] = True
                    maze_mask[void2a] = True
                    maze_mask[void2b] = True
                    maze_mask[end_void] = True

                future_branches += [child]

        current_branches = future_branches
        if history:
            maze_history = np.append(maze_history, maze_mask[None, ], axis=0)

    if history:
        return rendered_maze, maze_history
    else:
        return rendered_maze


def render_transition(maze, maze_history, fadein_frames=0, alpha_stop=0.1,
                      std=0.0, black=0.0, white=1.0, upfactor=None):
    """ Render the transition from white noise to maze.
    
    Args:
        maze (array_like): Final rendered maze
        maze_history (array_like): Frame-by-frame mask for the transition
        std (float, optional): Standard deviation of the background.
        black (float, optional): Value of the black color associated to walls.
        white (float, optional): Value of the white color associated to cells.
        fadein_frames (int, optional): Number of additional frames for fade in.
            Default is 0, meaning no fade in.
        alpha_stop (float optional): Percentage from where fade-in is considered finished.
        upfactor (int, optional): Upsampling factor for the maze. Default is None, meaning no upsampling.
        
    Returns:   
        numpy.ndarray: Rendered transition.
        
    """

    maze_frames = maze_history.shape[0]
    canvas_shape = (maze_frames + fadein_frames,) + maze.shape

    maze_mask = maze_history.astype(np.float)

    if fadein_frames < 0:
        raise ValueError("fadein_frames must be a non-negative integer")
    elif fadein_frames > 0:
        alpha_fadein = alpha_stop ** (1 / fadein_frames)

        final_mask = maze_mask[-1:, ]

        maze_mask = insert_frames(maze_mask, fadein_frames, pos=-1, ref=-1)

        maze_mask = ar_filter(maze_mask, alpha_fadein, axis=0)

        maze_mask[-1, ] = final_mask

    if upfactor is not None:
        upsampling = (1, upfactor, upfactor)
        maze_mask = resample(maze_mask, upsampling)
        maze_ = resample(maze, upfactor)
        bg = background(canvas_shape, white, black, std, upsampling=upsampling)
    else:
        maze_ = maze
        bg = background(canvas_shape, white, black, std)

    transition = (1 - maze_mask) * bg + maze_mask * maze_

    return transition


def fade_out(seq, alpha_stop, std=0.0, black=0.0, white=1.0, downsample=None):
    """ Make a video sequence vanish into background noise.
    
    Args:
        seq (array_like): Input sequence
        alpha_stop (float): Final attenuation
        std (float, optional): Standard deviation of the AR(1) process.
        black (float, optional): Value of the black color associated to walls.
        white (float, optional): Value of the white color associated to cells.
        downsample (int, optional): Downsampling factor.
            If set, frame size is downsampled prior to background noise generation
            and the generated background noise is consequently upsampled back to the original size.
    
    Returns: 
        numpy.ndarray: Faded out sequence.
        
    """
    seq_shape = seq.shape
    seq_len = seq_shape[0]
    seq_dims = len(seq_shape)

    if downsample is None:
        bg_shape = seq_shape
        upsampling = None
    else:
        bg_shape = [seq_len, ] + [s // downsample for s in seq_shape[1:]]
        upsampling = (1,) + (downsample, ) * (seq_dims - 1)

    alpha_fadeout = alpha_stop ** (1 / seq_len)

    bg_noise = background(bg_shape, white, black, std, upsampling=upsampling)

    alpha_seq = alpha_fadeout ** np.arange(seq_len).reshape((-1,) + (1,) * (seq_dims - 1))

    fadeout = alpha_seq * seq + (1 - alpha_seq) * bg_noise

    return fadeout


def shift_video(seq, alpha_shift, alpha_t, max_shift):
    """ Random shift of a video sequence. Shift is modeled as AR(1) along all dimensions.
    
    Args:
        seq (array_like): Input sequence.
        alpha_shift (float): Parameter of AR(1) across length and width.
        alpha_t (float): Parameter of AR(1) across frames.
        max_shift (float): Maximum allowed pixel shift.
    
    Returns:
        numpy.ndarray: Shifted video sequence.
        
    """
    n_frames, height, width = seq.shape

    std_shift = max_shift / 2 * ar_gain(alpha_t) * ar_gain(alpha_shift)

    def shift_vector(n):
        shift_vec = trunc_norm(loc=0, scale=std_shift, a=-2*std_shift, b=2*std_shift, size=(n_frames, n))
        shift_vec = ar_filter(shift_vec, alpha_t, axis=0)
        shift_vec = ar_filter(shift_vec, alpha_shift, axis=-1)
        shift_vec = np.round(np.clip(shift_vec, -max_shift, max_shift)).astype(np.int)
        return shift_vec

    shift_row = shift_vector(height)
    shift_col = shift_vector(width)

    shifted_seq = np.zeros_like(seq)

    for t in range(n_frames):
        frame = seq[t, ]
        for i in range(height):
            frame[i, ] = np.roll(frame[i, ], shift_row[t, i])
        for j in range(width):
            frame[:, j] = np.roll(frame[:, j], int(shift_col[t, j]))
        shifted_seq[t, ] = frame

    return shifted_seq


def insert_frames(array, n_frames, pos, ref=None, axis=0,
                  std=0.0, black=0.0, white=1.0, downsample=None):
    """ Insert frames into the maze sequence.
    
    Args:
        array (array_like): Input array containing video sequence.
        n_frames (int): Number of frames to insert.
        pos (int): Position where the frames should be inserted
        ref (int, optional): Index reference. If set, n_frames copies of the frame at ref are inserted.
            Otherwise background noise is generate.
        axis (int, optional): Axis into which frames are inserted
        std (float, optional): Standard deviation of the AR(1) process.
        black (float, optional): Value of the black color associated to walls.
        white (float, optional): Value of the white color associated to cells.
        downsample (float, optional): Downsampling factor.
            If set, frame size is downsampled prior to background noise generation
            and the generated background noise is consequently upsampled
            back to the original size.
    
    Returns:
        numpy.ndarray: Array with inserted frames.
        
    """
    array_shape = array.shape
    ax_len = array_shape[axis]
    position = pos % ax_len

    if ref is None:
        if downsample is None:
            bg_shape = tuple(n_frames if i == axis else dim for i, dim in enumerate(array_shape))
            upsampling = None
        else:
            bg_shape = tuple(n_frames if i == axis else dim // downsample for i, dim in enumerate(array_shape))
            upsampling = tuple(1 if i == axis else downsample for i, dim in enumerate(array_shape))

        frames = background(bg_shape, white, black, std, upsampling=upsampling)

    else:
        ref_frame = np.expand_dims(array.take(ref, axis=axis), axis=axis)
        frames = np.repeat(ref_frame, n_frames, axis=axis)

    pre = array.take(range(0, position), axis=axis)
    post = array.take(range(position, ax_len), axis=axis)

    new_array = np.concatenate((pre, frames, post), axis=axis)

    return new_array


def add_wgn(array, std):
    """ Add white Gaussian noise to array.
    
    Args:
        array (array_like): Input array
        std (float): Standard deviation of Gaussian noise.
    
    Returns:
        numpy.ndarray: Noisy copy of array.
        
    """
    return array + normal(scale=std, size=array.shape)


## Main program

Putting the blocks together.

### Constants

In [0]:
maze_width = 32
maze_height = 32
wall_thickness = 4

noise_std = 0.4
background_std = .25
white = 0.5
black = -0.5

fps = 12

sec_fadein = 0.5
sec_start = 0.5 - sec_fadein
sec_sustain = 1 - sec_fadein
sec_fadeout = 1

start_frames = time2samples(sec_start, fps)
fadein_frames = time2samples(sec_fadein, fps)
sustain_frames = time2samples(sec_sustain, fps)
end_frames = time2samples(sec_fadeout, fps)

max_shift = wall_thickness * 1.5
lag = wall_thickness * 4
att = mag2db(2 * max_shift)

alpha_maze = .5      # Do not set higher than .5
alpha_stop = .1
alpha_t = .8
alpha_shift = get_alpha(att, lag)

bg_args = {'std': background_std, 'black': black, 'white': white}  # Parameters for background generation

### Maze generation

I use Wilson's algorithm, but any maze generation algorithm should work.

In [0]:
print("Generating maze graph.")
root, children = wilson_algorithm(maze_height, maze_width)
print("Rendering basic maze.")
maze_bw = render_maze(root, children, alpha=0.99, std=0.0, history=False)  # Basic b&w rendering
frameshow(maze_bw)

### Rendering

In [0]:
print("Rendering AR maze")
maze, maze_history = render_maze(root, children, alpha_maze, **bg_args) # Actual AR rendering
maze_history = maze_history[::2] # Halve the size as it is too heavy otherwise


history_length = maze_history.shape[0]

sec_maze = history_length / fps
total_sec = sec_start + sec_fadein + sec_maze + sec_sustain + sec_fadeout
print(f"Building the maze takes {round(sec_maze, 1)} seconds @{fps}fps."
      f"Total duration is {round(total_sec, 1)} seconds.")

frameshow(maze)

In [0]:
print("Rendering transition from noise to maze. This may take a while")
bg2maze = render_transition(maze, maze_history, fadein_frames, alpha_stop,
                            **bg_args, upfactor=wall_thickness)
print("Show zoomed-in maze")
frameshow(bg2maze[-1,])

In [0]:
print("Show half-emerged maze")
frameshow(bg2maze[history_length // 2,])

In [0]:
print("Rendering transition from maze to noise.")
maze_appended = insert_frames(bg2maze, sustain_frames + end_frames, pos=-1, ref=-1)
maze_appended[-end_frames:, ] = fade_out(maze_appended[-end_frames:, ],
                                                      alpha_stop, downsample=wall_thickness, **bg_args)
print("Prepending noise frames.")
canvas = insert_frames(maze_appended, start_frames, pos=0, ref=None,
                       downsample=wall_thickness, **bg_args)
print(f"Done. Total number of frames is {canvas.shape[0]}")

In [0]:
print("Shifting maze.")
canvas_shifted = shift_video(canvas, alpha_shift, alpha_t, max_shift)

i_frame = start_frames + fadein_frames + history_length + sustain_frames // 2

frameshow(canvas_shifted[i_frame,])

In [0]:
print("Adding fine-grained noise.")
maze_fg = add_wgn(canvas_shifted, noise_std)
frameshow(maze_fg[i_frame, ])

### Animation

In [0]:
print("Plotting animation. This may take a while")
anim = animate(maze_fg, fps)
rc('animation', html='jshtml')
anim